<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/The_Pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
%load_ext google.colab.data_table
%tensorflow_version 2.x
from google.colab import drive #drive.flush_and_unmount()
from os import chdir
import numpy as np
import pandas as pd
drive.mount('/content/drive')
chdir('/content/drive/My Drive/CSIT Internship/code')

TensorFlow 2.x selected.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#pretty much taken from the args parameters defined in train.py
class extConfig():
    ext_ff_size = 2048
    ext_heads = 8
    ext_hidden_size = 768
    ext_layers = 2
    ext_dropout = 0.2
    large = False #whether to use 'bert-large-uncased' or 'bert-base-uncased' from pytorch-transformers
    temp_dir = '../temp' #useless
    finetune_bert = False #whether output of bert model is detached
    max_pos = 512
    param_init = 'nan' #shouldnt be used since we are loading from checkpoint

# Imports

In [0]:
try:
    already_run
except:
    already_run = False
if not already_run:
    !pip install --upgrade spacy
    #!python -m spacy download en_core_web_sm
    #!python -m spacy download en_core_web_md
    !python -m spacy download en_core_web_lg

    !pip install tensorboardX
    !pip install pyrouge
    !pip install pytorch-transformers
    !pip install transformers
    !pip install torch==1.1.0

    !python -m nltk.downloader wordnet
    !python -m nltk.downloader omw

    !pip install text2num
    
already_run = True

In [0]:
import spacy
from spacy import displacy
import en_core_web_lg
nlp = en_core_web_lg.load()

from transformers import BertTokenizer

import nltk
from nltk.corpus import wordnet as wn

import functools
from text_to_num import alpha2digit

In [0]:
chdir('/content/drive/My Drive/CSIT Internship/code/PreSumm/src')
import models
models.__path__.append('/content/drive/My Drive/CSIT Internship/code/PreSumm/src/models') #workaround for importing
import torch
import torch.nn as nn
from pytorch_transformers import BertModel

from models.encoder import ExtTransformerEncoder

# Classes

In [0]:
#taken from nlpyang/PreSumm/src/models/model_builder.py; minimally modified for a more applicative purpose
#actually seems as if model was trained on top of default BERT? cause theres only 1 checkpoint for both large & base

#obviously a wrapper. actually useless though.
class Bert(nn.Module):
    def __init__(self, large, temp_dir, finetune=False):
        super(Bert, self).__init__()
        if(large):
            self.model = BertModel.from_pretrained('bert-large-uncased', cache_dir=temp_dir)
        else:
            self.model = BertModel.from_pretrained('bert-base-uncased', cache_dir=temp_dir)

        self.finetune = finetune

    def forward(self, x, segs, mask):
        if(self.finetune):
            top_vec, _ = self.model(x, segs, attention_mask=mask)
        else:
            self.eval()
            with torch.no_grad():
                top_vec, _ = self.model(x, segs, attention_mask=mask)
        return top_vec

class ExtSummarizer(nn.Module):
    def __init__(self, args, device, checkpoint):
        super(ExtSummarizer, self).__init__()
        self.args = args
        self.device = device
        self.bert = Bert(args.large, args.temp_dir, args.finetune_bert)

        if(args.max_pos>512):
            my_pos_embeddings = nn.Embedding(args.max_pos, self.bert.model.config.hidden_size)
            my_pos_embeddings.weight.data[:512] = self.bert.model.embeddings.position_embeddings.weight.data
            my_pos_embeddings.weight.data[512:] = self.bert.model.embeddings.position_embeddings.weight.data[-1][None,:].repeat(args.max_pos-512,1)
            self.bert.model.embeddings.position_embeddings = my_pos_embeddings

        self.ext_layer = ExtTransformerEncoder(self.bert.model.config.hidden_size, args.ext_ff_size, args.ext_heads,
                                               args.ext_dropout, args.ext_layers)

        if checkpoint is not None:
            self.load_state_dict(checkpoint['model'], strict=True)
        '''
        else:
            if args.param_init != 0.0:
                for p in self.ext_layer.parameters():
                    p.data.uniform_(-args.param_init, args.param_init)
            if args.param_init_glorot:
                for p in self.ext_layer.parameters():
                    if p.dim() > 1:
                        xavier_uniform_(p)
        '''
        self.to(device)

    '''
    Lol some docscript practice time
    @param src one-hot tokens (padded)
    @param segs alternating type ids of 1 & 0 demarcating sentences (padded)
    @param clss indices of [CLS] tokens
    @param mask_src attention mask for src (ie 1 for everything, 0 for pads)
    @param mask_cls stateful mask of [CLS] tokens for training. Setting as all 1.
    '''
    #src = idxs, segs = alternating 1 & 0 type ids, clss = indices of [CLS] token
    def forward(self, src, segs, clss, mask_src, mask_cls):
        top_vec = self.bert(src, segs, mask_src)
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()
        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)
        return sent_scores, mask_cls

#My pipe components


In [0]:
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers', 'encoder', 'ff_actv', 'use_interval', 'rnn_size']
class sentence_scorer():

    def load_model(self,ckpt_path='../models/bertext_cnndm_transformer.pt',config=extConfig,device='cuda'):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.config = config()
        self.device = device
        checkpoint = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
        opt = vars(checkpoint['opt'])
        for k in opt.keys():
            if k in model_flags:
                setattr(self.config, k, opt[k])
        self.model = ExtSummarizer(self.config, self.device, checkpoint)

    def __call__(self,doc):
        src = []
        segs = []
        clss = []
        isZero = True

        for sent in doc.sents:
            enc = self.tokenizer.encode_plus(sent.text)
            if len(src) + len(enc['input_ids']) > self.config.max_pos: continue

            clss.append(len(src))
            src += enc['input_ids']

            if isZero: segs += [0] * len(enc['input_ids'])
            else: segs += [1] * len(enc['input_ids'])
            isZero = not isZero

        mask_cls = [1] * len(clss)

        inilen = len(src)
        src += [0]*(self.config.max_pos - inilen)
        segs += [0]*(self.config.max_pos - inilen)
        mask_src = [1]*inilen + [0]*(self.config.max_pos - inilen)
        
        src = torch.tensor([src],device=self.device)
        segs = torch.tensor([segs],device=self.device)
        clss = torch.tensor([clss],device=self.device)
        mask_src = torch.tensor([mask_src],device=self.device,dtype=torch.uint8)
        mask_cls = torch.tensor([mask_cls],device=self.device,dtype=torch.uint8)

        result = sorted(zip(self.model(src,segs,clss,mask_src,mask_cls)[0][0].detach().cpu().numpy(),doc.sents),reverse=True)
        
        doc.set_extension('sentence_scores',default=result,force=True)
        return doc

In [0]:
merge_nps = nlp.create_pipe('merge_noun_chunks')
class query_extractor():
    include_tokens = (
        'ADJ',
        'ADV',
        'NOUN',
        'PROPN',
        'VERB',
        'NUM', #prioritize numbers for cases where ambigious measurement
        'X' #assuming unrecognized is important in context of passage scanning
    )
    #excluding only those included by coarse POS
    exclude_tokens = (
        'ADD',
        'LS',
        'NIL',
        'WRB',
        'WP',
        'WP$',
        'PRP'
    )
    #https://nlp.stanford.edu/software/dependencies_manual.pdf
    #https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md
    exclude_dependency = ()

    @functools.lru_cache(maxsize=256, typed=False)
    def convert2noun(self,word):    
        synsets = wn.synsets(word)
        if len(synsets) == 0: return None
        lemmas = [l for s in synsets for l in s.lemmas()]
        derivationally_related_forms = [(l, l.derivationally_related_forms()) for l in lemmas]
        nouns = [l.name() for drf in derivationally_related_forms for l in drf[1] if l.synset().name().split('.')[1] == wn.NOUN]
        nouns += [l.name() for l in lemmas if l.synset().name().split('.')[1] == wn.NOUN]
        result = [(w, float(nouns.count(w))/len(nouns)) for w in set(nouns)]
        result.sort(key=lambda w: -w[1])
        if len(result) == 0: return None
        #print(result)
        return result[0][0]

    #exists for conciseness
    max_nouns = 3
    max_query = 5

    #BROKEN UP INTO FUNCTIONS FOR EASY IMPROVEMENT IN FUTURE

    def check_throw(self,t):
        return not t.is_oov and (t.pos_ not in self.include_tokens or t.tag_ in self.exclude_tokens or t.is_stop)

    def assign_keyness(self,sent):
        for t in sent: t.set_extension('magnitude',getter=lambda t:np.linalg.norm(t.vector,ord=1),force=True) #np.inf for max, ord=2 for euclidean, ord=1 for manhattan #manhattan most suitable as all 300 dimensions should be perpendicular in nature

    def assign_nouns(self,names,nouns):
        ans = names[:self.max_nouns]
        if len(names) < self.max_nouns:
            nouns.sort(key=lambda t: len(t.text.split())/len(t.text)) #longer words but shorter noun phrases likely more important
            ans += nouns[:self.max_nouns-len(names)]
        return ans

    def convert_noun(self,sent):
        for t in sent: t.set_extension('noun_form',getter=lambda t:self.convert2noun(t.text),force=True)

    def score_token(self,t):
        if t.is_oov: return 999
        if t.pos_ == 'NUM': return 999
        if t.is_currency: return 999
        return t._.magnitude

    def __call__(self,doc):
        queries = []
        docc = merge_nps(doc)
        for sent,msent in zip(doc.sents,docc.sents):
                       
            keep = []
            names = []
            nouns = []

            for t in list(sent) + list(msent):
                if self.check_throw(t): continue
                if t.pos_ == 'PROPN': names.append(t)
                elif t.pos_ == 'NOUN': nouns.append(t)
                else: keep.append(t)
            
            #remove sentences with no proper nouns or that only contain them
            if len(names) == len(sent) or len(names) == 0: queries.append('')
            else:
                names = list(set(names))
                nouns = list(set(nouns))
                keep = list(set(keep))
                
                query = self.assign_nouns(names,nouns)

                #naive sentiment by means of word vector
                self.assign_keyness(keep)
                keep.sort(key=self.score_token,reverse=True)
                query += keep

                proc = set()

                #Temporarily 'Cancelled"
                #VVV
                #TODO: better discrimination of what to convert
                self.convert_noun(query[:self.max_query])


                for t in query[:self.max_query]:
                    if not t.pos_ in ('PROPN','NOUN','NUM','X'):
                        if t._.noun_form: proc.add(t._.noun_form)
                        else: proc.add(t.text)
                        #proc.add(t.text)
                    else: proc.add(t.text)
                
                queries.append(' '.join(list(proc)))
                #print(list(zip(keep,[t._.magnitude for t in keep])))
                #print(dict((t,(t.dep_,t.head)) for t in query))
        doc.set_extension('queries',default=list(zip(doc.sents,queries)),force=True)
        return doc

#Pipeline

In [8]:
if False and not nlp.has_pipe('merge_noun_chunks'):
    merge_nps = nlp.create_pipe('merge_noun_chunks')
    nlp.add_pipe(merge_nps,after='parser')

if not nlp.has_pipe('merge_entities'):
    merge_ents = nlp.create_pipe('merge_entities')
    nlp.add_pipe(merge_ents,after='ner')

extractor = query_extractor()
if nlp.has_pipe('query_extractor'):
    nlp.replace_pipe('query_extractor',extractor)
else:
    nlp.add_pipe(extractor, last=True,name='query_extractor')

scorer = sentence_scorer()
scorer.load_model()
if nlp.has_pipe('sentence_scorer'):
    nlp.replace_pipe('sentence_scorer',scorer)
else:
    nlp.add_pipe(scorer, last=True,name='sentence_scorer')
#train textcat to determine if sentence is a claim or similar for checking?
import gc
gc.collect()
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f78a2f6fe10>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f78a2dd0348>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f78a2dd03a8>),
 ('merge_entities', <function spacy.pipeline.functions.merge_entities>),
 ('query_extractor', <__main__.query_extractor at 0x7f77f4afb320>),
 ('sentence_scorer', <__main__.sentence_scorer at 0x7f77f4afb6d8>)]

In [9]:
#very liberal essay courtesy of me after a culture studies course
test1 = '''It is undeniable that Singapore has enjoyed great peace in regards to matters of race, especially compared to other nations such as the United States which currently remain polarised on the matter. Nonetheless, I think that rather than these issues not existing, they were suppressed very well, festering unnoticed. In this essay I will be reflecting on an example of discrimination in Singapore that I know of, and why and why not they may be acceptable. 
	While idealists like me dream of a day where such a word is meaningless except in the annals of History, for now, and many years to come, race still holds great significance. Why does physical appearance, our skin colour, matter so much? The answer, it does not. Rather, it happened to correlate to other features, features collectively known as Ethnicity, due to Geographical and Historical factors. And it continues to propagate its significance through stereotyping and active polarising influences. 
	Let us start with one of the most prominent things that correlates with race, tradition, a subset of the Culture associated with different races. Tradition is one of the common attributes that can draw people together. While the education system has been trying to get people of different races of understand each others Culture through initiatives such as Racial Harmony Day, it cannot be denied that on a normal basis one would better understand a custom their family (mostly homogenous in Singapore, will explain later) forces them to undergo rather than that of another race. This however creates an unseen privilege in regards to this. Objectively, there are quite a few disruptive Chinese traditions, such as the Hungry Ghost Festival (which can be described as “artificial haze”) or Lion Dances (very loud). I am not sure about other races, in itself an illustration of the above point, but I have heard that other racial Cultures similarly have loud traditions. Hence it can be generalized that such historical and hence racially separated tradition tends to be noisy. 
	What happens now? Well, Singapore is about three quarters Chinese. A Chinese is best able to appreciate their own historical Culture compared to those of other races. What this means is such traditional rites being carried out publically appear perfectly fine to the majority (by which I mean Chinese) in Singapore. However, what about those of other races? Due to the lack of appreciation for their culture, and the rareness of it (it is not accepted as a fact of life), their traditional events may be considered disruptive or even silly by the majority. And because there is a lack of understanding in the first place, close friends are not able to dispel the misconceptions, and may even become influenced. This is can be a polarising force. 
	Even worse, Singapore’s initiatives at conserving our heritages have turned into initiatives to conserve Chinese heritage. Take for example the scale of the Speak Mandarin movement. To most Chinese, it would not even come to mind that this is out of the ordinary. Ask one and they probably would not know about initiatives for any other races’ Culture. All of this is very insidious. It may not appear to have effects, but it creates a hotbed for small mistakes and misconceptions that can spiral into disasters. Naturally, there will be individuals who believe in strongly in their stereotypes, but in an age where communication is so easy, such stereotypes can spread more easily than ever before, no longer restrained by temporal limits, let alone restrained by awareness.
	However, the situation in Singapore has yet to become a full blown disaster, due to the emphasis placed on other dividing characteristics such as those of Meritocracy. However, look no further than the United States to find a worst case scenario. Already sabotaged historically, the racial polarisation in United States only grows worse due to multiplicity of polarised cultural groups, rampant stereotypes of those groups and general atmosphere of chaos and anger, encouraged by a free media that values sensationalism. Physical appearance there is both so correlated and associated with ethnicity due to the polarisation resulting in tribalistic instincts emerging, and often that container of ethnicity is not light, and depending on your “alignment” carries many strong stereotypes both good and bad. However, for Singapore to get from here to that, it would be near impossible. On its own, Singaporean society values ability and money too much to get caught up with it, and are on average too well educated to easily accept stereotypes. So what would it take?
	Getting to my policy suggestion, I believe Singapore should cease its emphasis on our cultural heritage. No more Speak Mandarin Movement as one example. That is a very bold statement to make, with equally bold reasons why. Firstly, Culture is not something that is governed, it is something that governs itself. Maybe some cultural analysts think what is happening in Singapore is cultural dilution, but I think not. Let cultural fusion happen, even if it borders on cultural appropriation, because what it allows for is a real shared heritage everyone can appreciate. Even when this seemingly results in cultural homogeneity, it can still be vibrant in relation to the rest of the world, and is much better than having another racial riot. Race is a very divisive thing due to all its historical baggage and resulting constructs, such dilution can serve to weaken the concept of race, creating new Culture along lines that are not race. And that is what youth today naturally gravitate towards. In the ages of seeking one’s own identity, but also in this age of information saturation and cultural dilution which allows for greater understanding of it, youth no longer define their identity along race as heavily anymore. However, initiatives such as the Speak Mandarin Movement go counter to this. They are designed to force youths to define their identity along their ethnicity, and consequently their race. By placing too heavy an emphasis on the Historical baggage, it will only result in majority of people mainly seeing each other by their race again, and even worse, push the ever changing field of culture to remain divided along race, and possibly even push it into the sort of vicious cycles of racial identity as seen in other nations. 
	Secondly, we should encourage youths to create their own culture, from a very young age. Our education system could have children in primary school play games related to the nature of culture. They could be encouraged to come up with their own tribes with made up customs, once in a while swapping the groups of course, and have pseudo-conflicts to better ingrain the nature of culture into youths so they do not make the same mistake of placing emphasis on things (race for example) that have been overemphasized for too long.
	Of course, with the current direction Singapore is going, the conservative nature of the majority here, all of this will be hard to execute. There are many people who genuinely believe it is right, for no reason, to continue our cultural heritage by forcing it to remain exactly the same and passed down. I do not believe in such things as there being a right culture, and think that practically, the best solution to prevent racial polarisation is simply cultural fusion. Naturally, youths are heading in that direction, but there are also many being pushed to remain conservative on culture, and mainly strongly refrain from even thinking about a topic regarding as Out of Bounds. It is impossible and wrong to get people to simply abandon their existing culture and force them to adapt to change just because it would seem more efficient. It is the preference of many not to change after all, and so massive change should not be the solution. 
	Moving forwards I do not think Singapore will ever descend into complete chaos due to race before we are eliminated by something else, like for example a meteor strike (aka it will not happen). Our historical and geographical situations and limitations means we will not fall down that rabbit hole. However, it is possible Singapore will become a bit more aware of the racial differences, which in itself is a bad thing if not followed up by appreciation of other ethnic cultures. It is already happening after all.
	Regarding the course, I found it more of a mentally stimulating activity. They were things I learnt, such as better expression in regards to these areas and the proper terminologies. I also had misconceptions around terms such as gender being dispelled. I have also become more sensitive to the existences of the various femininities and masculinities, and hence can exploit code switching better. Race-wise, I have learnt more about how the situation in the United States came to be. However, I have always believe in the day that physical appearance will no longer matter as part of one’s identity, and personally strive towards the conditions to create such a world. In that regards, being more aware of racial and sexual divides has allowed me to understand better the conditions that lead to them and gives me room for thought on how to mitigate them.
'''
with nlp.disable_pipes('sentence_scorer'):
    doc1 = nlp(alpha2digit(test1,'en'))
[s[1] for s in doc1._.queries if s[1] != '']

['Singapore the United States polarization matters compare',
 'Singapore muser discrimination acceptability an example',
 'correlation start different races tradition the Culture',
 'Singapore Culture undergo homogeny Racial Harmony Day',
 'the Hungry Ghost Festival artificial haze quite a few disruptive Chinese traditions loudness Lion Dances',
 'Singapore Chinese about 3 quarters',
 'appreciator A Chinese other races compare their own historical Culture',
 'Singapore Chinese fineness such traditional rites publically',
 'mind most Chinese coming ordinary',
 'Singapore Meritocracy other dividing characteristics place',
 'the United States look a worst case scenario discovery',
 'sensationalism growth value multiplicity United States',
 'Singapore impossibleness approach',
 'Singapore getting end our cultural heritage my policy suggestion',
 'No more Speak Mandarin Movement one example',
 'Singapore some cultural analysts thought materialisation cultural dilution',
 'weakener resulting

In [10]:
#https://www.straitstimes.com/world/united-states/chinese-restaurants-in-the-us-are-closing-and-thats-a-good-thing-the-owners-say
test2 = ''' More than 40 years after buying Eng's, a Chinese-American restaurant in the Hudson Valley, Tom Sit is reluctantly considering retirement.  For much of his life, Mr Sit has worked here seven days a week, 12 hours a day. He cooks in the same kitchen where he worked as a young immigrant from China. He parks in the same space where he'd take breaks and read his wife's letters, sent from Montreal while they courted by post in the late 1970s. He seats his regulars at the same tables where his three daughters did homework.  Two years ago, at the insistence of his wife, Mrs Faye Lee Sit, he started taking off one day a week. Still, it's not sustainable. He's 76, and they're going to be grandparents soon. Working 80 hours a week is just too hard.  But his grown daughters, who have college degrees and good-paying jobs, don't intend to take over.  Across the country, owners of Chinese-American restaurants like Eng's are ready to retire but have no one to pass the business to. Their children, educated and raised in America, are pursuing professional careers that do not demand the same gruelling labor as food service.  According to new data from the restaurant reviewing website Yelp, the share of Chinese restaurants in the top 20 metropolitan areas has been consistently falling. Five years ago, an average of 7.3 per cent of all restaurants in these areas were Chinese, compared with 6.5 per cent today. That reflects 1,200 fewer Chinese restaurants at a time when these 20 places added more than 15,000 restaurants overall.  Even in San Francisco, home to the oldest Chinatown in the United States, the share of Chinese restaurants shrank to 8.8 per cent from 10 per cent.  It doesn't seem that interest in the cuisine has faltered. On Yelp, the average share of page views of Chinese restaurants hasn't declined, nor has the average rating.  And at the same time, the percentages of Indian, Korean and Vietnamese restaurants - many of which were also owned and operated by immigrants from Asian countries - are holding steady or increasing nationwide.  The restaurant business has always been tough, and rising rents and delivery apps haven't helped. Tightening regulations on immigration and accounting have also made it harder for cash-based restaurants to do business.  But those are not factors specific to Chinese restaurants, and do not explain the wave of closings. Instead, a big reason seems to be the economic mobility of the second generation.  "It's a success that these restaurants are closing," said Ms Jennifer Lee, a former New York Times journalist who wrote of the rise of Chinese restaurants in her book The Fortune Cookie Chronicles and produced a documentary The Search For General Tso.  "These people came to cook so their children wouldn't have to, and now their children don't have to."  The retirements of the restaurant owners also reflect the history of Chinese immigration to the United States. In 1882, the Chinese Exclusion Act halted what had been a steady rise in people coming from China. It was not revoked until 1943, and large-scale immigration resumed only after 1965, when other race-targeting quotas were abolished.  China's Cultural Revolution, an often-violent social and political upheaval that started in 1966, prompted many young people to emigrate to the US, a country that projected an image of freedom and economic possibility.  Mr Sit left Guangzhou, in southern China, in 1968. He hiked, climbed and swam his way to Hong Kong, filling his pants with pine cones as an improvised flotation device.  "There was just no future," he said. "The only way to get freedom and to get a good job was to go to Hong Kong."  In 1974, he immigrated to the US and started working at Eng's, which opened in 1927. He had never worked in a restaurant before, but the heat from the woks was much less intense than what he experienced at a Hong Kong plastics factory where he had worked.  Unlike Mr Sit, some immigrants had been chefs in China. They served Hunan and Cantonese foods on linen tablecloths to bejewelled, curious diners at places like Shun Lee Palace in New York.  "There was the golden age of Chinese cooking in America, starting in the late 1960s and early 1970s," said Mr Ed Schoenfeld, a restaurateur and chef who had worked in Chinese restaurants since the 1970s. "We started getting regional practitioners of fine regional cuisine to come to this country and do their thing."  Mostly, though, the newly minted chefs cooked quickly and cheaply. They adapted their method of cooking to American tastes, developing dishes like beef chow fun, fortune cookies and egg drop soup, often brought home in the signature takeout containers.  "They were not precious," Ms Lee said. "These people did not come to be chefs; they came to be immigrants, and cooking was the way they made a living."  Other immigrant groups follow a similar pattern. With social mobility and inclusion in more mainstream parts of the economy, the children of immigrants are less likely than their parents to own their own businesses.  "In some ways, the children are regaining the status of the first generation that they have lost while migrating," said sociology professor Jennifer Lee of Columbia University and co-author of The Asian American Achievement Paradox. (She is not related to Ms Lee, the journalist.) "The goal has never been to continue those businesses."  When they do become entrepreneurs, these children tend to work in industries like tech or consulting, rather than in food service or nail salons.  In the past decade, some members of the second generation have also chosen to take charge of family restaurants. Nom Wah Tea Parlor, a New York dim sum restaurant that opened in 1920, has stayed a family business: first run by the Choy family, then the Tangs.  The 41-year-old owner, Mr Wilson Tang, left a career in finance to succeed his uncle in 2011. Initially, his parents baulked at his decision.  "As immigrants, it's the only thing you can do; if it's not restaurants, it's a laundromat," he said. "For me to choose to go back to owning a restaurant? That was tough for them to accept."  Since then, Nom Wah has expanded - to another Manhattan location, to Philadelphia and to Shenzhen, China. On any given night, groups of guests wait for a table outside the Chinatown location for up to an hour, huddled in the bend of Doyers Street.  "I had this unique opportunity to preserve something that was from old New York," Mr Tang said. "I still work extremely hard. But I also know how to use marketing tools, like the Internet."  In a parallel effort, the team behind Junzi Kitchen, a fast-casual Chinese restaurant chain based in New York, recently raised US$5 million (S$6.7 million) to research and buy places like Eng's, rebranding them with Junzi's modern take on the cuisine.  "They are still going to have their usual beloved Chinese takeout services, but we are providing an upgraded version of that," said Mr Yong Zhao, the founder and chief executive.  But family-run Chinese restaurants are typically not being passed to the next generation. Some may close up shop, sell their businesses to other first-generation immigrants or move on and see their former storefronts become something else entirely.  Mr Sit has not yet found the right person to run the restaurant, and has no immediate plans to close.  "To take over Eng's, you have to keep the heart in Eng's," he said. "You need to have a loyalty to the business, not just someone who thinks, 'I'll make one year, two years of money, I don't care.'"  Mrs Sit feels more ready to retire than her husband. Normally talkative, he can be evasive whenever the family tries to bring up a successor.  "They'll have to work hard," she said, her eyes sparkling as she teased her husband. "Like Tom Sit. Maybe then he'll let them take over."  If he ever actually does hand Eng's to someone else, Mr Sit will miss his customers, and miss running an operation.  But he is proud of what he built. He is proud that his daughters, American-born educated professionals, are working in jobs they have chosen, jobs they love.  "I hoped they have a better life than me," he said. "A good life. And they do."'''
with nlp.disable_pipes('query_extractor'):
    doc2 = nlp(alpha2digit(test2,'en'))
doc2._.sentence_scores

[(0.44611275,
  More than 40 years after buying Eng's, a Chinese-American restaurant in the Hudson Valley, Tom Sit is reluctantly considering retirement.  ),
 (0.31025773,
  Across the country, owners of Chinese-American restaurants like Eng's are ready to retire but have no one to pass the business to.),
 (0.15926246,
  According to new data from the restaurant reviewing website Yelp, the share of Chinese restaurants in the top 20 metropolitan areas has been consistently falling.),
 (0.14281066,
  5 years ago, an average of 7.3 per cent of all restaurants in these areas were Chinese, compared with 6.5 per cent today.),
 (0.0868622,
  On Yelp, the average share of page views of Chinese restaurants hasn't declined, nor has the average rating.  ),
 (0.08647902,
  Even in San Francisco, home to the oldest Chinatown in the United States, the share of Chinese restaurants shrank to 8.8 per cent from 10 per cent.  ),
 (0.08198352,
  But his grown daughters, who have college degrees and good-p

In [12]:
#https://demo.allennlp.org/open-information-extraction
#!pip install allennlp
#OEI seems promising, but doesn't seem to reliably generate coherent sentences.
#^Preferring coherence over simplicity

     |████████████████████████████████| 7.6MB 4.9MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 245kB 73.0MB/s 
     |████████████████████████████████| 30.9MB 100kB/s 
     |████████████████████████████████| 734.6MB 24kB/s 
     |████████████████████████████████| 256kB 60.0MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 163kB 55.9MB/s 
     |████████████████████████████████| 133kB 37.1MB/s 
     |████████████████████████████████| 3.2MB 46.7MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
     |████████████████████████████████| 2.1MB 45.3MB/s 
  Created wheel for ftfy: filename=ftfy-5.6-cp36-none-any.whl size=44553 sha256=09322f072e27794686174ff7308d7d5d74b4bb708e3ecbd438c1b9518c5f982d
  Stored in directory: /root/.cache/pip/wheels/43/34/ce/cbb38d71543c408de56f3c5e26ce8ba495a0fa5a28eaaf1046
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5